<a href="https://colab.research.google.com/github/5dogs/scrapingProductsWeb/blob/newbranch/%E8%A3%BD%E5%93%81%E6%83%85%E5%A0%B1ScrapingToGspread%E9%80%A3%E6%90%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GSpread APIの認証情報(JSONファイル形式)を←のメニューバーからアップロードする。多分毎回のランタイム接続ごとにやり直さないといけない？otherwise,GSpreadと連携できない

In [ ]:
import requests
import gspread
from bs4 import BeautifulSoup
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials
from google.oauth2 import service_account


SP_CREDENTIAL_FILE= "/content/GSpread連携withGcolab.json"


SP_COPE=[
    "https://www.googleapis.com/auth/drive",
    "https://spreadsheets.google.com/feeds"
]
credentials=service_account.Credentials.from_service_account_file(SP_CREDENTIAL_FILE, scopes=SP_COPE)

#ServiceAccountCredentials.from_json_keyfile_name(SP_CREDENTIAL_FILE,SP_COPE)
gc = gspread.authorize(credentials)
worksheet = gc.open('バイトてすと').sheet1

In [ ]:
def pickURLs(j):
  column = "F"
  rowwww = j
  url = worksheet.acell( f"{column}{rowwww}").value
  return url

In [ ]:
from PIL import Image
import io
import os
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

def saveImg(img,file_name):
  target_folder_id = "1p3ccAEI-UVP9gVXQVim_m2Y72bQF47Ek"
  folder_name = "2023/03/18石川ぬんた"

  folder_metadata = {
    'name': folder_name,
    'parents': [target_folder_id],
    # contentType
    'mimeType': 'application/vnd.google-apps.folder'}
  # driveAPIアクセスのための認証情報
  service = build('drive', 'v3', credentials=credentials)

  folderData = service.files().create(body=folder_metadata, fields='id, webViewLink').execute()
  folder_id  = folderData.get('id')
  folder_url = folderData.get('webViewLink')
  folder_url_str = str(folder_url)
  #ファイルを見たいときは、、、webContentLink
  print(folder_id)
  print("これfolder_url→",folder_url)

  # file_name = "製品番号テスト用567.png"
  # 画像ファイルを親フォルダに保存する
  folderWithfile = os.path.join(folder_name, file_name)
  if not os.path.exists(folder_name):
      os.makedirs(folder_name)
  img.save(folderWithfile)
  print(folderWithfile)


  file_metadata = {
    'name': file_name,
    'parents': [folder_id]}
  # folderWithfileのアップロード
  media = MediaFileUpload(folderWithfile, resumable=True)
  fileData = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
  file_id = fileData.get('id')
  # file_url = fileData.get('webContentLink')
  print('file_id: %s' % file_id)
  # print(file_url)
  return folder_url_str

# URLから画像の取得・変換と保存
# url = "https://www.settsu.co.jp/catalogs/sersrc/1824"
def pickImg_fromURL__convert_toPng(url):
  html = requests.get(url).content
  soup = BeautifulSoup(html, "html.parser")
  img_src = soup.find('li', {'id': 'main-image-0'}).find('img')['src']
  print(img_src)
  img_url = "https://www.settsu.co.jp"+img_src
  response = requests.get(img_url)
  img = Image.open(io.BytesIO(response.content))
  return img

# saveImg(pickImg_fromURL__convert_toPng(url))

In [ ]:
def pickFeatureList():
  end_col = worksheet.row_values(1).index('（項目）')  # "項目"の列のインデックスを取得する
  values = worksheet.range(1, 14, 1, end_col)
  print(end_col)

  featureList = []
  for cell in values:
      if cell.col == end_col+1:
          break
      featureList.append(cell.value)

  return featureList
pickFeatureList()

20


['材質・色', '主要材質', '色・外装処理', '電磁シールド', '防音性能', '静止荷重', '耐震性能']

In [ ]:
url = "https://www.settsu.co.jp/catalogs/sersrc/1960"

def pickSpantag(url):
  response = requests.get(url)
  html = response.text

  # BeautifulSoupを使用してHTMLを解析
  soup = BeautifulSoup(html, "html.parser")

  # <span>タグを抜き出して、その中のテキストを抜き出す
  span_tags = soup.find_all("span")

  text_list = []
  for tag in span_tags:
      if tag.get("class") == ["line1"] or tag.get("class") == ["line2"]:
          print(tag.text.strip())
          text_list.append(tag.text.strip())
  text_str = "".join(text_list)
  return text_str
pickSpantag(url)

配線ラック
SSRW シリーズ


'配線ラックSSRW シリーズ'

In [ ]:
def pickInfo(featureList,url):
  featureDataList= []
  # print("以下が!!!!!!!!",url)
  html = requests.get(url).content
  soup = BeautifulSoup(html, "html.parser")
  table = soup.find("table")
  table_tag = soup.find('div', {'class': 'feature-table'})
  trs = table_tag.findAll("tr")

  for feature in featureList:
    found = False
    for tr in trs:
      if feature in tr.text:
        found = True
        tds = tr.findAll("td")
        for td in tds:
          cleaned_text = td.text.replace('\n', '').replace('　', '')
          # \¥uが王道らしいけどもねー

          featureDataList.append(cleaned_text)
        break
    if not found:
      # print("NULL")
      featureDataList.append("NULL")
  return featureDataList

In [ ]:
def pickComment(url):
  html = requests.get(url).content
  soup = BeautifulSoup(html, "html.parser")
  comment = soup.find('p', class_='series-feature-comment')
  comment = comment.text.replace('\n', '').replace(' ', '')
  return comment

In [ ]:
def puttableData(info):
  i = 0
  while i<len(info):

    worksheet.update_cell(k, 14 + i, info[i])
    print(info[i])

    i = i +1
  return

In [ ]:
def putData(info1,info2,info3,info4):

  worksheet.update_cell(k, 9, info1)
  worksheet.update_cell(k, 8, info2)
  worksheet.update_cell(k, 11, info3)
  worksheet.update_cell(k, 10, info4)

In [ ]:
setList =pickFeatureList()
k = 25
while True:
  url = pickURLs(k)
  if url is None:
    break
  info1 = pickComment(url)
  info2 = pickSpantag(url)

  file_name = info2+".png"
  info3 = saveImg(pickImg_fromURL__convert_toPng(url),file_name)
  info4 = file_name

  putData(info1, info2, info3, info4)
  puttableData(pickInfo(setList,url))

  k = k +1

NameError: ignored